# Cifar-10 이미지 분류
### 전체 데이터 사용
### Data Augmentation 적용
### 출처: [Achieving 90% accuracy in Object Recognition ](https://appliedmachinelearning.blog/2018/03/24/achieving-90-accuracy-in-object-recognition-task-on-cifar-10-dataset-with-keras-convolutional-neural-networks/)

In [177]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.layers import Activation, BatchNormalization
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.regularizers import l2
import numpy as np
import os

#### 자료형 변환 및 스케일링
- X: 실수형으로 정규화
- Y: 1-hot encoding
    * airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck

In [178]:
(X_train, y_train0), (X_test, y_test0) = cifar10.load_data()
print(X_train.shape, X_train.dtype)
print(y_train0.shape, y_train0.dtype)
print(X_test.shape, X_test.dtype)
print(y_test0.shape, y_test0.dtype)

(50000, 32, 32, 3) uint8
(50000, 1) uint8
(10000, 32, 32, 3) uint8
(10000, 1) uint8


In [179]:
X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0

X_test = X_train[10000:12500]
X_train = X_train[0:10000]
y_test0 = y_train0[10000:12500]
y_train0 = y_train0[0:10000]

X_train.shape, X_test.shape, y_train0.shape, y_test0.shape

((10000, 32, 32, 3), (2500, 32, 32, 3), (10000, 1), (2500, 1))

In [180]:
Y_train = tf.keras.utils.to_categorical(y_train0, 10)
Y_test = tf.keras.utils.to_categorical(y_test0, 10)
Y_train[:4]

array([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]], dtype=float32)

#### 모형 구현

In [181]:
np.random.seed(0)
weight_decay = 1e-4

In [182]:
model = Sequential()

model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay), 
                 input_shape=X_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
# model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=l2(weight_decay)))
# model.add(Activation('elu'))
# model.add(BatchNormalization())
# model.add(MaxPooling2D(pool_size=(2,2)))
# model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_62 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
activation_56 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_56 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_57 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_57 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 16, 16, 32)      

In [183]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='categorical_crossentropy', 
              optimizer=RMSprop(lr=0.001, decay=weight_decay), 
              metrics=['accuracy'])

In [184]:
# 모델 최적화 설정
MODEL_DIR = './model/'
if not os.path.exists(MODEL_DIR):
    os.mkdir(MODEL_DIR)

In [185]:
modelpath = MODEL_DIR + "Cifar10-{epoch:02d}-{val_loss:.4f}.hdf5"
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', 
                               verbose=1, save_best_only=True)

In [186]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    if epoch > 40:
        lrate = 0.0003
    return lrate

In [187]:
#data augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(X_train)

In [188]:
from tensorflow.keras.callbacks import LearningRateScheduler

history = model.fit_generator(datagen.flow(X_train, Y_train, batch_size=64),
                    steps_per_epoch=X_train.shape[0] // 64, epochs=10,
                    verbose=2, validation_data=(X_test, Y_test),
                    callbacks=[LearningRateScheduler(lr_schedule), checkpointer])

  ...
    to  
  ['...']
Train for 156 steps, validate on 2500 samples
Epoch 1/10

Epoch 00001: val_loss improved from inf to 3.02927, saving model to ./model/Cifar10-01-3.0293.hdf5
156/156 - 25s - loss: 2.5948 - accuracy: 0.2992 - val_loss: 3.0293 - val_accuracy: 0.1232
Epoch 2/10

Epoch 00002: val_loss improved from 3.02927 to 2.16234, saving model to ./model/Cifar10-02-2.1623.hdf5
156/156 - 24s - loss: 2.0279 - accuracy: 0.3868 - val_loss: 2.1623 - val_accuracy: 0.3248
Epoch 3/10

Epoch 00003: val_loss improved from 2.16234 to 1.57604, saving model to ./model/Cifar10-03-1.5760.hdf5
156/156 - 24s - loss: 1.7787 - accuracy: 0.4374 - val_loss: 1.5760 - val_accuracy: 0.4688
Epoch 4/10

Epoch 00004: val_loss improved from 1.57604 to 1.35931, saving model to ./model/Cifar10-04-1.3593.hdf5
156/156 - 24s - loss: 1.6156 - accuracy: 0.4811 - val_loss: 1.3593 - val_accuracy: 0.5460
Epoch 5/10

Epoch 00005: val_loss improved from 1.35931 to 1.34851, saving model to ./model/Cifar10-05-1.3485.hdf

#### 모델 평가하기

In [189]:
from tensorflow.keras.models import load_model
del model
model = load_model('model/Cifar10-10-1.0756.hdf5')

In [190]:
# 테스트 정확도 출력 
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, Y_test, verbose=0)[1]))


 Test Accuracy: 0.6452
